In [1]:
include("datasetloader.jl")
include("train.jl")

random_cutoff (generic function with 1 method)

In [2]:
const PACKAGE_PATH = "."
M = Model(joinpath(PACKAGE_PATH, "ngrams"))

Model(["ara", "bel", "ben", "bul", "cat", "ces", "dan", "deu", "ell", "eng"  …  "spa", "swa", "swe", "tat", "tgl", "tur", "ukr", "vie", "yid", "zho"], Dict("epo" => 11, "jpn" => 24, "kor" => 26, "ina" => 21, "ron" => 38, "deu" => 8, "nds" => 33, "fin" => 13, "kab" => 25, "bel" => 2…), Dict{Vector{UInt8}, Float32}[Dict([0xa7, 0xd9, 0x84, 0xd9, 0x82, 0xd8, 0xb7] => -11.61864, [0x84, 0xd9, 0x83, 0x20, 0xd8, 0xa3, 0xd9] => -12.067797, [0x84, 0xd8, 0xb5, 0x20, 0xd8] => -12.370045, [0x83, 0xd9, 0x85, 0xd8, 0xa7] => -10.843754, [0x20, 0xd8, 0xab, 0xd8, 0xa7, 0xd9, 0x86] => -12.4390135, [0xd8, 0xb1, 0xd8, 0xa9, 0x20, 0xd8, 0xa8] => -11.741152, [0xd8, 0xa7, 0xd8, 0xac, 0xd8, 0xb9] => -12.633021, [0x81, 0xd9, 0x8a, 0xd9, 0x87, 0xd8] => -11.471882, [0xa7, 0x20, 0xd8, 0xb7] => -11.985819, [0xd8, 0xb4, 0xd8, 0xb1, 0xd8, 0xb7] => -11.70128…), Dict([0xb0, 0xd1, 0x86, 0xd1, 0x8b, 0xd1, 0x8f] => -10.577563, [0xd0, 0xbf, 0xd0, 0xbb, 0xd0, 0xb0, 0xd1] => -11.394588, [0x91, 0xd0, 0xb4, 0xd1] => -11.630513

In [3]:
# for Q in M.Qs
#     for(k,v) in zip(keys(Q), shuffle(values(Q)|>collect))
#         Q[k] = v
#     end
# end

In [ ]:
using BSON
using SampledVectors
ckp = BSON.load("model.bson")
M = ckp[:M]
train_loss = ckp[:train_loss]
default_q_list = ckp[:default_q_list];

In [3]:
WDs = [WikiDataSet("corpus/wikipedia/test", langs=[l]) for l in M.languages]
TDs = [TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=[l]) for l in M.languages]
datasets = [WDs; TDs]
val_set = BatchedLoader(WeightedLoader(datasets, weights=log.(length.(datasets))), 8)

BatchedLoader(WeightedLoader(Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}[Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}(Base.Iterators.Cycle{RandomLoader}(RandomLoader(["آمونبرغ  هي بلدة، مدينة و بلدة ألمانية تقع في ألمانيا في Marburg-Biedenkopf. بحاجة لمصدر \nمدن شتولبن، Château-Garnier، تورو سول تراسيمينو و Tragwein هي مدن توأمة لـآمونبرغ.\nهذه بذرة مقالة عن موقع جغرافي في هسن بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آني بيدويل \u200f هي سفرجات أمريكية، ولدت في 1839، وتوفيت في 1918. 2  3  4 \nهذه بذرة مقالة عن عالم نبات أمريكي بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آنا هيوم \u200f ؛ مترجمة، كاتِبة وشاعرة بريطانية. 1  2  3 \nهذه بذرة مقالة عن  شاعر بريطاني بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آني لافلور \u200f ؛ كاتِبة وشاعرة كندية. 1  2 \nهذه بذرة مقالة عن  شاعر كندي بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آيت يعقوب هو دُوَّار يقع

In [5]:
WDs = [WikiDataSet("corpus/wikipedia/train", langs=[l]) for l in M.languages]
TDs = [TatoebaDataset("corpus/tatoeba", "tatoeba_train.txt", langs=[l]) for l in M.languages]
datasets = [WDs; TDs]
train_set = BatchedLoader(WeightedLoader(datasets, weights=log.(length.(datasets))), 8)

BatchedLoader(WeightedLoader(Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}[Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}(Base.Iterators.Cycle{RandomLoader}(RandomLoader(["آب\u200cبادامو  هي قرية تقع في قسم جايدشت الريفي  في إيران. 1  يقدر عدد سكانها بـ 0 نسمة بحسب إحصاء 2016. 2 \nهذه بذرة مقالة عن موقع جغرافي في محافظة فارس بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آتوس سوانتس  هو مبارز بالسيف برازيلي، مثّل بلاده في الألعاب الأولمبية الصيفية 2012. 1 \nآتوس سوانتس على موقع FIE \nهذه بذرة مقالة عن مبارز سيف شيش برازيلي بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "تعديل - تعديل مصدري - تعديل ويكي بياناتآخر أيام الأرض فيلم صوتي درامي من تأليف وإخراج خالد المهدي وبطولة حنان ترك وخالد صالح وهاني عادل وخالد النبوي وعمرو القاضي. 1  2  3 \nتدور أحداث الفيلم عن التصور الإسلامي لنهاية العالم، حيث يروي قصة 40 يوما من عمر البشرية تحفل بالأحداث المصيرية وفي مقدمتها المعركة الف

In [7]:
include("train.jl")

l, grad = loss_and_grad(M, first(val_set), 5)
# step!(M, grad)
# M.default_q, l

(0.033261888730970246, (Qs = Dict{Any, Any}[Dict(UInt8[0x20, 0x70, 0x72] => 1.010815f-6, UInt8[0x75, 0x66, 0x66, 0x65] => -5.9805657f-6, UInt8[0x63, 0x72, 0x65] => -5.980297f-6, UInt8[0x63, 0x73, 0x20] => 5.0552774f-7, UInt8[0x65, 0x73, 0x6e, 0x20] => -1.0479776f-5, UInt8[0x74, 0x20, 0x75] => 5.055214f-7, UInt8[0x73, 0x20, 0x74] => -3.2401207f-5, UInt8[0x6c, 0x79, 0x20, 0x69, 0x6e] => -1.0479996f-5, UInt8[0x74, 0x20, 0x67, 0x65] => -1.047993f-5, UInt8[0x61, 0x73, 0x73, 0x20] => -1.04799465f-5…), Dict(UInt8[0x98, 0xe6, 0xb4] => -5.055374f-7, UInt8[0xe9, 0x9d, 0xa2, 0x20, 0xe6] => -7.282298f-7, UInt8[0xaf, 0xe7, 0x9a] => -1.9294947f-7, UInt8[0x90, 0x84, 0xe6] => -5.0553643f-7, UInt8[0x96, 0xe8, 0x80, 0x85] => -1.6725994f-7, UInt8[0x87, 0x20, 0xe6, 0x98] => -5.3527515f-7, UInt8[0xe6, 0xac, 0xa1, 0xe7] => -1.9295307f-7, UInt8[0xac, 0xe6, 0x9c] => -5.055306f-7, UInt8[0xa6, 0x96, 0xe4] => -1.6726013f-7, UInt8[0x98, 0x20] => -1.5127273f-6…)], default_q = 0.017604344f0))

In [32]:
Zygote.forwarddiff

forwarddiff (generic function with 1 method)

In [22]:
val_set = BatchedLoader(WeightedLoader(datasets, weights=log.(length.(datasets))), 8)

BatchedLoader(WeightedLoader(Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}[Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}(Base.Iterators.Cycle{RandomLoader}(RandomLoader(["آمونبرغ  هي بلدة، مدينة و بلدة ألمانية تقع في ألمانيا في Marburg-Biedenkopf. بحاجة لمصدر \nمدن شتولبن، Château-Garnier، تورو سول تراسيمينو و Tragwein هي مدن توأمة لـآمونبرغ.\nهذه بذرة مقالة عن موقع جغرافي في هسن بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آني بيدويل \u200f هي سفرجات أمريكية، ولدت في 1839، وتوفيت في 1918. 2  3  4 \nهذه بذرة مقالة عن عالم نبات أمريكي بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آنا هيوم \u200f ؛ مترجمة، كاتِبة وشاعرة بريطانية. 1  2  3 \nهذه بذرة مقالة عن  شاعر بريطاني بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آني لافلور \u200f ؛ كاتِبة وشاعرة كندية. 1  2 \nهذه بذرة مقالة عن  شاعر كندي بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آيت يعقوب هو دُوَّار يقع

In [58]:
function loss_and_grad3(params, p_ys)
    grad = gradient(params) do params
        Zygote.forwarddiff([params]) do params
            get_loss(params, p_ys)
        end
    end
    grad
end
# function loss_and_grad3(params, p_ys)
#     grad = withgradient(params) do params
#             get_loss(params, p_ys)
#         end
#     grad
# end
function loss_and_grad3(model::Model, data, ngram)
    params = (Qs=model.Qs, default_q=model.default_q, cutoff_list=model.cutoff_list)
    # params_aux = ()
    data = preprocess_data(data, ngram, M.langs_inds::Dict{String, Int})
    loss_and_grad3(params, data)
end

loss_and_grad3 (generic function with 2 methods)

In [71]:
include("train.jl")

random_cutoff (generic function with 1 method)

In [4]:
data = first(val_set)
# data = preprocess_data(data, M.ngram, M.langs_inds)
# d1 = data[1]

8-element Vector{Pair{String, String}}:
                                                                                                                                                                        "Мин - яңа укытучы." => "tat"
                                                                                                                                     "Itu adalah pemandandan tercantik yang pernah kulihat." => "msa"
                                                                                                                                                                         "Миңа киңәш кирәк." => "tat"
                                                                                                                                                     "Melhor nem imaginar uma coisa dessas." => "por"
                                                                                                  "Kuulin että Tomilla ja Marilla meni sukset ristiin ja Mari on nyt vap

In [6]:
using JET
@report_opt loss_and_grad9(M, data, 5)

═════ 185 possible errors found ═════
┌ loss_and_grad9(model::Model, data::Vector{Pair{String, String}}, ngram::Int64) @ Main /home/guoyongzhi/langid_expirement/train.jl:88
│┌ loss_and_grad8(params::NamedTuple{(:Qs, :default_q), Tuple{Vector{Dict{Vector{UInt8}, Float32}}, Float32}}, params_aux::NamedTuple{(:cutoff_list,), Tuple{Vector{Float32}}}, p_ys::Vector{Tuple{Dict{Vector{UInt8}, Float32}, Vector{Float32}}}) @ Main /home/guoyongzhi/langid_expirement/train.jl:79
││┌ withgradient(f::var"#69#70"{NamedTuple{(:cutoff_list,), Tuple{Vector{Float32}}}, Vector{Tuple{Dict{Vector{UInt8}, Float32}, Vector{Float32}}}}, args::NamedTuple{(:Qs, :default_q), Tuple{Vector{Dict{Vector{UInt8}, Float32}}, Float32}}) @ Zygote /home/guoyongzhi/.julia/packages/Zygote/4rucm/src/compiler/interface.jl:154
│││┌ pullback(f::var"#69#70"{NamedTuple{(:cutoff_list,), Tuple{Vector{Float32}}}, Vector{Tuple{Dict{Vector{UInt8}, Float32}, Vector{Float32}}}}, args::NamedTuple{(:Qs, :default_q), Tuple{Vector{Dict{Vector

In [62]:
loss_and_grad(M, data, 5)

(1.8551428003626516, (Qs = Dict{Any, Any}[Dict(UInt8[0xa6] => 9.183378f-8, UInt8[0x61, 0x64] => 1.4102098f-7, UInt8[0x69, 0x61] => 3.0328215f-6, UInt8[0x6e, 0x65] => 2.9282317f-6, UInt8[0x6f, 0x6d] => 5.2971f-6, UInt8[0x70, 0x20] => 1.7716835f-7, UInt8[0x63, 0x68] => 2.115316f-7, UInt8[0x6e, 0x69] => 5.1504817f-6, UInt8[0x69, 0x6e] => 8.501434f-6, UInt8[0x72, 0x65] => 1.313366f-6…), Dict(UInt8[0x20, 0x74, 0x68, 0x65] => 2.103362f-6, UInt8[0x69, 0x61] => 2.5490383f-6, UInt8[0x63, 0x68] => 1.7917463f-7, UInt8[0x6e, 0x69] => 4.581064f-6, UInt8[0x69, 0x6e] => 7.3958736f-6, UInt8[0x20, 0x74, 0x68] => 2.1033593f-6, UInt8[0x61] => 2.3203444f-5, UInt8[0x72, 0x65] => 1.0886114f-6, UInt8[0x73, 0x20] => 1.6336418f-6, UInt8[0x72, 0x69] => 4.390228f-7…), Dict(UInt8[0xa6] => 1.2036944f-7, UInt8[0xb6] => 2.2725534f-7, UInt8[0xa4] => 2.5119922f-7, UInt8[0x99] => 1.1331902f-6, UInt8[0x9a, 0x20] => 1.2586207f-7, UInt8[0xa1] => 1.4199578f-6, UInt8[0x95] => 1.6316819f-6, UInt8[0x9b] => 1.258296f-7, UInt8[

In [69]:
@time data = first(val_set);
@time get_loss(M, first(val_set), M.langs_inds)
@time l, grad = loss_and_grad(M, data, 5)
@time step!(M, grad)

  0.034085 seconds (105 allocations: 7.422 KiB)


  0.079193 seconds (22.86 k allocations: 1.791 MiB)


  3.904617 seconds (26.20 M allocations: 791.085 MiB, 41.10% gc time)


  0.135301 seconds (701.20 k allocations: 16.903 MiB)


In [30]:
@time data = first(val_set);
@time get_loss(M, first(val_set), M.langs_inds)
@time l, grad = loss_and_grad(M, data, 5)
@time step!(M, grad)

  0.021757 seconds (126 allocations: 7.062 KiB)


  0.065987 seconds (8.01 k allocations: 748.844 KiB)


  0.734107 seconds (4.76 M allocations: 124.865 MiB, 33.02% gc time)
  0.021206 seconds (112.20 k allocations: 2.055 MiB)


In [28]:
@time data = first(val_set);
@time get_loss(M, first(val_set), M.langs_inds)
@time l, grad = loss_and_grad(M, data, 5)
@time step!(M, grad)

  0.048674 seconds (162 allocations: 13.320 KiB)


  0.271632 seconds (72.97 k allocations: 6.413 MiB)


  1.910263 seconds (14.75 M allocations: 373.793 MiB, 34.36% gc time)
  0.044178 seconds (298.16 k allocations: 5.460 MiB)


In [6]:
using Plots
using SampledVectors
using BSON
function plot_series(names_values...; title=nothing, path=".")
    fig = plot()
    if title === nothing
        title = only(names_values)[1]
    end
    title!(fig, title)
    y_m = Inf
    y_M = -Inf
    colors = Plots.palette(:Dark2_5)
    for (i,(name, value)) in enumerate(names_values)
        isempty(value) && continue
        y = sampled(value)
        if !(eltype(y) <: Real)
            x = first.(y)
            y = last.(y)
        else
            x = collect(sampledindexes(value))
        end
        fig = plot!(x, y, label="", linecolor=alphacolor(colors[i], 0.3))
        ylims!(fig, quantile(y, (0.005, 0.995)))
        step = max(1, length(y) ÷ 100)
        y_ = mean.(Iterators.partition(y, step))
        x_ = x[1:step:end]
        fig = plot!(x_, y_, label=name, linecolor=colors[i], linewidth=1.5)
        m, M = extrema(y_)
        y_m = min(y_m, m)
        y_M = max(y_M, M)
    end
    savefig("$path/$(title).png")
    extr = (y_m, y_M)
    if sum(abs.(extr .- ylims(fig))) > only(diff(collect(extr)))
        title!(fig, "$title zoom")
        ylims!(fig, extr)
        savefig("$path/$(title)-zoom.png")
    end
end

plot_series (generic function with 1 method)

In [75]:
plot_series("train_loss" => train_loss, "val_loss" => val_loss, title="loss")

In [9]:
train_loss = SampledVector{Float64}(4096)
val_loss = SampledVector{Tuple{Int, Float64}}(4096)
default_q_list = SampledVector{Float64}(4096)

0-element SampledVector{Float64, Int64, Int64, Int64, Int64}

In [12]:
ckp = BSON.load("model.bson")
M = ckp[:M]
train_loss = ckp[:train_loss]
default_q_list = ckp[:default_q_list];

In [13]:
get_loss(M, first(val_set))

0.0011204660908108455

In [ ]:
mkpath("trainlogs")
for i in 1:10000
    l, grad = loss_and_grad(M, first(train_set), rand([1:M.ngram; 3:M.ngram; 3:min(5, M.ngram)]))
    step!(M, grad, 0.1f0)
    push!(train_loss, l)
    push!(default_q_list, M.default_q)
    iters = length(train_loss)
    if iters % 10 == 0
        l_val = get_loss(M, first(val_set))
        push!(val_loss, (length(train_loss), l_val))
    end
    if iters % 100 == 0
        plot_series("train_loss" => train_loss, "val_loss" => val_loss, title="loss", path="trainlogs")
        plot_series("default_q" => default_q_list, path="trainlogs")
    end
    if iters % 100 == 0
        random_cutoff(M)
    end
end
bson("model.bson", M=M, train_loss=train_loss, val_loss=val_loss, default_q_list=default_q_list)
length(train_loss)


In [342]:
@time random_cutoff(M)

  0.045683 seconds (400.05 k allocations: 20.140 MiB)


7